# Logistic Regression

In [1]:
import scipy as sp
import numpy as np
import pandas as pd

In [2]:
train_df = pd.read_csv(
    '../lab01/nyc-taxi-trip-duration/cleaned_train.csv'
)

In [265]:
class LogisticRegression:
    def __init__(
        self,
        penalty='l2',
        tol=1e-4,
        C=1.0,
        fit_intercept=True,
        max_iter=100
    ):
        if penalty not in ['l1', 'l2', None]:
            raise ValueError("Wrong penalty: '%s'" % penalty)
        self.penalty = penalty
        
        if not isinstance(tol, (int, float)):
            raise TypeError("Wrond tol type: '%s'" % type(tol))
        self.tol = tol
        
        if not isinstance(C, (int, float)):
            raise TypeError("Wrond C type: '%s'" % type(C))
        self.C = C
        
        if not isinstance(fit_intercept, bool):
            raise TypeError("Wrond fit_intercept type: '%s'" % type(fit_intercept))
        self.fit_intercept = fit_intercept
        
        if not isinstance(max_iter, int):
            raise TypeError("Wrond max_iter type: '%s'" % type(max_iter))
        self.max_iter = max_iter
    
    def __get_l1_penalty(self):
        def l1_penalty(w):
            return 1/self.C * np.abs(w)
        
        def der_l1_penalty(w):
            # ignoring zeros existence
            return 1/self.C * ((w > 0) * 1 + (w <= 0) * -1)
        
        return l1_penalty, der_l1_penalty
    
    def __get_l2_penalty(self):
        def l2_penalty(w):
            return 1/self.C * np.multiply(w, w)
        
        def der_l2_penalty(w):
            return 2/self.C * w
        
        return l2_penalty, der_l2_penalty
    
    def __get_None_penalty(self):
        return None, None
    
    def fit(self, X, y, debug=False):
        assert \
            len(X.shape) == 2, \
            "X should be 2D vector"
        assert \
            y.shape == (X.shape[0], 1), \
            "y should be 2D vector and should correspond to X"
        
        _X = X.copy()
        _y = y.copy()
        
        if self.fit_intercept:
            _X = np.hstack((
                _X, 
                np.ones(
                    (X.shape[0], 1)
                )
            ))
        
        args = [_X, _y]
        
        args.extend(
            getattr(
                self,
                '_LogisticRegression__get_' + str(self.penalty) + '_penalty'
            )()
        )
        
        self.w = np.ones((_X.shape[1], 1))
        
        if debug:
            return args
        
        result = sp.optimize.minimize(
            self.__cost,
            self.w,
            args,
            'L-BFGS-B',
            self.__gradient,
            tol=self.tol,
            options={
                'maxiter': self.max_iter
            }
        )
        
        assert result.success, result.message
        
        self.w = result.x
    
    @staticmethod
    def __predict(X, w):
        def predict_real(x, w):
            return x @ w

        def sigmoid(z):
            return 1 / (1 + np.exp(-z))
        
        return sigmoid(predict_real(X, w))
    
    @staticmethod
    def __cost(w, args):
        X, y, penalty, _ = args
        
        predictions = LogisticRegression.__predict(X, w)
        
        m = X.shape[0]
        
        cost0 = -(1 - y).T @ np.log(1 - predictions)
        cost1 = -y.T @ np.log(predictions)
        
        penalty_part = penalty(w).sum() if penalty else 0
        
        final_cost = (cost0 + cost1).sum() / m + penalty_part
        
        return final_cost
    
    def predict(self, X):
        if self.fit_intercept:
            _X = np.hstack((
                X, 
                np.ones(
                    (X.shape[0], 1)
                )
            ))
        return self.__predict(_X, self.w)
    
    @staticmethod
    def __gradient(w, args):
        X, y, _, der_penalty = args
        _w = w.reshape((-1, 1))
        
        predictions = LogisticRegression.__predict(X, _w)
        
        penalty_part = der_penalty(_w) if der_penalty else 0
        
        return X.T @ (predictions - y) + penalty_part

In [281]:
from sklearn.datasets import make_classification
from sklearn.metrics import roc_auc_score
import unittest

def dummy_dataset():
    X, y = make_classification(100, 20)
    y = y.reshape((100, 1))
    return X, y

def prepare(debug=True, penalty=None):
    X, y = dummy_dataset()

    lr = LogisticRegression(penalty=penalty)

    args = lr.fit(X, y, debug)
    
    return lr, X, y, args

class TestLogisticRegression(unittest.TestCase):
    def test_gradient(self):
        lr, X, y, args = prepare()
        
        self.assertEqual(
            lr._LogisticRegression__gradient(lr.w, args).shape,
            (21, 1)
        )
        
    def test_cost(self):
        lr, X, y, args = prepare()
        
        self.assertEqual(
            type(lr._LogisticRegression__cost(lr.w, args)),
            np.float64
        )
    
    def test_None(self):
        lr, X, y, args = prepare(False)
        score = roc_auc_score(y, lr.predict(X))
        print("Score: {}".format(score))
        
    def test_l1(self):
        lr, X, y, args = prepare(False, 'l1')
        score = roc_auc_score(y, lr.predict(X))
        print("Score: {}".format(score))
    
    def test_l2(self):
        lr, X, y, args = prepare(False, 'l2')
        score = roc_auc_score(y, lr.predict(X))
        print("Score: {}".format(score))

In [282]:
unittest.main(argv=['first-arg-is-ignored', '--verbose'], exit=False)

test_None (__main__.TestLogisticRegression) ... /home/a3nippo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:119: RuntimeWarning: divide by zero encountered in log
/home/a3nippo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:119: RuntimeWarning: invalid value encountered in matmul
ok
test_cost (__main__.TestLogisticRegression) ... ok
test_gradient (__main__.TestLogisticRegression) ... ok
test_l1 (__main__.TestLogisticRegression) ... ok
test_l2 (__main__.TestLogisticRegression) ... 

Score: 1.0
Score: 0.9648


FAIL

FAIL: test_l2 (__main__.TestLogisticRegression)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-281-30bff43b0f2f>", line 47, in test_l2
    lr, X, y, args = prepare(False, 'l2')
  File "<ipython-input-281-30bff43b0f2f>", line 15, in prepare
    args = lr.fit(X, y, debug)
  File "<ipython-input-265-e0c401980730>", line 97, in fit
    assert result.success, result.message
AssertionError: b'ABNORMAL_TERMINATION_IN_LNSRCH'

----------------------------------------------------------------------
Ran 5 tests in 0.053s

FAILED (failures=1)


Sometimes this happens due to too large l2 normalization.